In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow_gpu==2.0.0-beta1 tensorflow-datasets matplotlib tqdm tensorflow_hub scipy

import os


from data_generator import DataGenerator

import matplotlib.pyplot as plt

from numpy import array
import numpy as np


import datetime as dt
import datetime
from datetime import datetime

from tensorflow.keras.layers import Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences


from tensorflow.keras import datasets, layers, models

import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow as tf

%load_ext tensorboard
print(tf.__version__)

2.0.0-beta1


In [2]:
dt = DataGenerator(random=False, first_index=10)
dt.rewind()
dt.steps

431169

In [4]:
# integer encode input data
def onehot_encoded (integer_encoded, char_to_int = 2):
    # one hot encode
    onehot_encoded = list()
    letter = [0 for _ in range(char_to_int)]
    letter[integer_encoded] = 1
    onehot_encoded.append(letter)
    
    return onehot_encoded[0]


In [5]:
last_price = 0
last_time = 0
should_buy = 0
should_sell = 0

def get_date(state):
    timestamp = int(state["timestamp"])
    return datetime.fromtimestamp(timestamp)


def get_parse_state(raw_state, last_price, last_time):
    list = []
    price = raw_state["price"]
        
    list.append(raw_state["amount"])

    def prepare_orders(orders, price, multi):
        amount = float(orders[0][1])
        for order in orders:
            list.append((float(order[0])/price) * multi)
            list.append(float(order[1])/amount)

    history_step = 5
    bids = raw_state["bids"][:history_step]
    asks = raw_state["asks"][:history_step]
    prepare_orders(bids, price, 1)
    prepare_orders(asks, price, -1)

    if last_price != 0:
        list.extend([price/last_price])
    else:
        list.extend([0])
        
    current_timestamp = int(raw_state['timestamp'])
    if last_time != 0:
        list.extend([current_timestamp / last_time])
    else:
        list.extend([0])
    #list = []
    #list.extend([raw_state['timestamp']])
    #print(current_timestamp)
    return list

def get_future_state(df, sec=120): # 2 min in future
    state_timestamp = int(dt.get_from_index(dt.index-1)['timestamp'])
    timestamp_limit = state_timestamp + sec
    #print("Current timestamp", state_timestamp, " ==== ", timestamp_limit)
    index = 0
    timestamp = 0
    while timestamp < timestamp_limit:
        index += 3
        state = dt.get_from_index(dt.index + index)
        timestamp = int(state['timestamp'])
    return state



def get_state(raw_state):
    global last_price
    global last_time
    global should_buy
    global should_sell
    
    list = get_parse_state(raw_state, last_price, last_time)

    furure_state = get_future_state(dt)
    future_price = furure_state["price"]
    
    current_price = raw_state["price"]
    current_timestamp = int(raw_state['timestamp'])
    
    best_bid = float(furure_state["bids"][0][0]) 
    is_value_incresed = best_bid >= (current_price + 0.2)

    if is_value_incresed:
        should_buy += 1
        #print (current_price, " ==== ", (current_price + 0.2), " ===== ", furure_state)
        y = onehot_encoded(0)
    else:
        #print (current_price, " ==== ", (current_price + 0.2), " ===== ", furure_state)
        should_sell += 1
        y = onehot_encoded(1)
           
    #print (y)
    #print (get_date(raw_state), " ==== ", get_date(furure_state))
    
    last_price = current_price
    last_time = current_timestamp
    return [list, y]

In [6]:
from tqdm import tqdm

dt.rewind()

def get_set(data_count):
  trainX = []
  trainY = []
  for i in tqdm(range(data_count)):
      state = dt.next()
      state = get_state(state)
      trainX.append(state[0])
      trainY.append(state[1])
  trainX = np.array(trainX)
  trainY = np.array(trainY)
  return trainX, trainY

data_count = dt.steps - 1000

#data_count = 10000

val_percentage = 0.1

trainX, trainY = get_set(int(data_count*(1-val_percentage)))

valX, valY = get_set(int(data_count*val_percentage))

print(trainX.shape)
print(trainY.shape)


print(valX.shape)
print(valY.shape)


100%|██████████| 43016/43016 [01:00<00:00, 712.33it/s]

(387152, 23)
(387152, 2)
(43016, 23)
(43016, 2)


In [11]:
# define generator
n_input = 100
def get_gen(set_x, set_y):
  return TimeseriesGenerator(set_x, set_y, length=n_input, batch_size=128*2)

train_generator = get_gen(trainX, trainY)

val_generator = get_gen(valX, valY)

# number of samples
print('Samples: %d' % len(train_generator))

x, y = train_generator[0]

features = x.shape[2]

Samples: 1512


In [12]:
inputX = Input(shape=(n_input,features))

def add_deep_layers(input_layer):
    x = Dense(100)(input_layer)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x_shorcut = x
    x = Dense(100)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    x = Dense(100)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    x = Dense(100)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = layers.add([x, x_shorcut])
    return x

x = add_deep_layers(LSTM(features*40)(inputX))
x = add_deep_layers(x)
x = add_deep_layers(x)
x = add_deep_layers(x)
x = Dense(100)(x)
x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
x = Dense(100)(x)
x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
x = Dense(2, activation='softmax')(x)

model = Model(inputs=[inputX], outputs=x)


filepath = "stock.h5"


model.compile(optimizer=tf.keras.optimizers.Adagrad(lr=0.001, epsilon=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [14]:
#tf.keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

In [ ]:


checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = [checkpoint]

history = model.fit_generator(train_generator, 
                              validation_data=val_generator, 
                              validation_steps=len(val_generator)-1,
                              steps_per_epoch=len(train_generator)-1, 
                              epochs=35, verbose=1, 
                              callbacks=callbacks_list)


W0814 17:11:49.927427 140588483127104 deprecation.py:323] From /home/samir/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/35
1510/1511 [============================>.] - ETA: 4s - loss: 0.6309 - accuracy: 0.6725
Epoch 00001: loss improved from inf to 0.63086, saving model to stock.h5
1511/1511 [==============================] - 6433s 4s/step - loss: 0.6308 - accuracy: 0.6726 - val_loss: 0.5987 - val_accuracy: 0.7182
Epoch 2/35
1510/1511 [============================>.] - ETA: 4s - loss: 0.6246 - accuracy: 0.6735
Epoch 00002: loss improved from 0.63086 to 0.62454, saving model to stock.h5
1511/1511 [==============================] - 6409s 4s/step - loss: 0.6245 - accuracy: 0.6735 - val_loss: 0.6004 - val_accuracy: 0.7182
Epoch 3/35
1510/1511 [============================>.] - ETA: 4s - loss: 0.6220 - accuracy: 0.6738
Epoch 00003: loss improved from 0.62454 to 0.62196, saving model to stock.h5
1511/1511 [==============================] - 6409s 4s/step - loss: 0.6219 - accuracy: 0.6739 - val_loss: 0.5986 - val_accuracy: 0.7182
Epoch 4/35
1510/1511 [============================>.] - ETA: 4s - loss: 0.

In [ ]:

# Plot training & validation loss values
plt.plot(history.history['loss'], history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['accuracy'], history.history['val_accuracy'])
plt.title('Model loss')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
import datetime
print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))